# ఔట్‌పుట్ పార్సర్స్ బేసిక్స్

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో ఔట్‌పుట్ పార్సర్స్ గురించి నేర్చుకుంటాము. ఔట్‌పుట్ పార్సర్స్ అనేవి LLM నుండి వచ్చే అన్‌స్ట్రక్చర్డ్ ఔట్‌పుట్‌ని స్ట్రక్చర్డ్ ఫార్మాట్‌లోకి మార్చడానికి ఉపయోగపడే కంపోనెంట్స్.

## సెటప్

మొదట, మనం అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేసుకుందాం:

In [ ]:
# అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేయడం
!pip install langchain langchain-openai openai pydantic

ఇప్పుడు, మనం OpenAI API కీని సెట్ చేద్దాం:

In [ ]:
import os
from dotenv import load_dotenv

# .env ఫైల్ నుండి API కీని లోడ్ చేయడం
load_dotenv()

# లేదా డైరెక్ట్‌గా సెట్ చేయడం (డెవలప్‌మెంట్ కోసం మాత్రమే, ప్రొడక్షన్‌లో ఉపయోగించవద్దు)
# os.environ["OPENAI_API_KEY"] = "మీ-API-కీ-ఇక్కడ-పెట్టండి"

## ఔట్‌పుట్ పార్సర్స్ అంటే ఏమిటి?

ఔట్‌పుట్ పార్సర్స్ అనేవి LLM నుండి వచ్చే అన్‌స్ట్రక్చర్డ్ టెక్స్ట్ ఔట్‌పుట్‌ని స్ట్రక్చర్డ్ ఫార్మాట్‌లోకి (JSON, లిస్ట్‌లు, డిక్షనరీలు, మొదలైనవి) మార్చడానికి ఉపయోగపడే కంపోనెంట్స్.

ఔట్‌పుట్ పార్సర్స్ ఎందుకు ముఖ్యమైనవి?
1. **స్ట్రక్చర్డ్ డేటా**: LLM ఔట్‌పుట్‌ని ప్రోగ్రామాటిక్‌గా యాక్సెస్ చేయగల ఫార్మాట్‌లోకి మార్చడం
2. **కన్సిస్టెన్సీ**: LLM ఔట్‌పుట్‌ని కన్సిస్టెంట్ ఫార్మాట్‌లో పొందడం
3. **ఇంటిగ్రేషన్**: LLM ఔట్‌పుట్‌ని ఇతర సిస్టమ్స్‌తో సులభంగా ఇంటిగ్రేట్ చేయడం
4. **వాలిడేషన్**: LLM ఔట్‌పుట్ కరెక్ట్ ఫార్మాట్‌లో ఉందని నిర్ధారించడం

## LangChain లో ఔట్‌పుట్ పార్సర్స్

LangChain లో అనేక రకాల ఔట్‌పుట్ పార్సర్స్ ఉన్నాయి. వాటిలో కొన్నిటిని చూద్దాం:

### 1. StrOutputParser

`StrOutputParser` అనేది అత్యంత సింపుల్ ఔట్‌పుట్ పార్సర్, ఇది LLM ఔట్‌పుట్‌ని స్ట్రింగ్‌గా మార్చుతుంది:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = ChatPromptTemplate.from_template("తెలుగులో {topic} గురించి ఒక చిన్న వ్యాసం రాయండి.")

# StrOutputParser క్రియేట్ చేయడం
output_parser = StrOutputParser()

# చెయిన్ క్రియేట్ చేయడం
chain = prompt | llm | output_parser

# చెయిన్‌ని రన్ చేయడం
result = chain.invoke({"topic": "IPL క్రికెట్"})
print("StrOutputParser రిజల్ట్:")
print(f"టైప్: {type(result)}")
print(f"కంటెంట్: {result[:200]}...")

### 2. ListOutputParser

`ListOutputParser` అనేది LLM ఔట్‌పుట్‌ని లిస్ట్‌గా మార్చుతుంది:

In [ ]:
from langchain_core.output_parsers import ListOutputParser

# ListOutputParser క్రియేట్ చేయడం
list_parser = ListOutputParser()

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
list_prompt = ChatPromptTemplate.from_template(
    """తెలుగు సినిమాలలో టాప్ 5 హీరోల పేర్లను జాబితాగా అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: ప్రతి హీరో పేరును కొత్త లైన్‌లో రాయండి. ప్రతి పేరు ముందు నంబర్ లేదా బుల్లెట్ పాయింట్ పెట్టవద్దు.
    """
)

# చెయిన్ క్రియేట్ చేయడం
list_chain = list_prompt | llm | list_parser

# చెయిన్‌ని రన్ చేయడం
list_result = list_chain.invoke({})
print("ListOutputParser రిజల్ట్:")
print(f"టైప్: {type(list_result)}")
print(f"కంటెంట్: {list_result}")

# లిస్ట్‌లోని ఐటెమ్‌లను యాక్సెస్ చేయడం
print("\nలిస్ట్ ఐటెమ్‌లు:")
for i, hero in enumerate(list_result):
    print(f"{i+1}. {hero}")

### 3. CommaSeparatedListOutputParser

`CommaSeparatedListOutputParser` అనేది LLM ఔట్‌పుట్‌ని కామాతో విడదీయబడిన లిస్ట్‌గా మార్చుతుంది:

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

# CommaSeparatedListOutputParser క్రియేట్ చేయడం
comma_parser = CommaSeparatedListOutputParser()

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
comma_prompt = ChatPromptTemplate.from_template(
    """తెలుగు రాష్ట్రంలోని టాప్ 5 పర్యాటక ప్రదేశాల పేర్లను అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: ప్రతి ప్రదేశం పేరును కామాతో విడదీయండి. ఉదాహరణకు: ప్రదేశం1, ప్రదేశం2, ప్రదేశం3
    """
)

# చెయిన్ క్రియేట్ చేయడం
comma_chain = comma_prompt | llm | comma_parser

# చెయిన్‌ని రన్ చేయడం
comma_result = comma_chain.invoke({})
print("CommaSeparatedListOutputParser రిజల్ట్:")
print(f"టైప్: {type(comma_result)}")
print(f"కంటెంట్: {comma_result}")

# లిస్ట్‌లోని ఐటెమ్‌లను యాక్సెస్ చేయడం
print("\nలిస్ట్ ఐటెమ్‌లు:")
for i, place in enumerate(comma_result):
    print(f"{i+1}. {place}")

### 4. JsonOutputParser

`JsonOutputParser` అనేది LLM ఔట్‌పుట్‌ని JSON ఫార్మాట్‌లోకి మార్చుతుంది:

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

# JsonOutputParser క్రియేట్ చేయడం
json_parser = JsonOutputParser()

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
json_prompt = ChatPromptTemplate.from_template(
    """తెలుగు సినిమా {movie_name} గురించి సమాచారం అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫీల్డ్స్‌తో:
    - title: సినిమా పేరు
    - year: విడుదల సంవత్సరం (నంబర్)
    - director: దర్శకుడు పేరు
    - actors: ప్రధాన నటుల జాబితా (అరే)
    - plot: కథ సారాంశం
    - rating: మీ రేటింగ్ 1-10 స్కేల్‌లో (నంబర్)
    """
)

# చెయిన్ క్రియేట్ చేయడం
json_chain = json_prompt | llm | json_parser

# చెయిన్‌ని రన్ చేయడం
json_result = json_chain.invoke({"movie_name": "RRR"})
print("JsonOutputParser రిజల్ట్:")
print(f"టైప్: {type(json_result)}")
print(f"కంటెంట్: {json_result}")

# JSON ఫీల్డ్స్‌ని యాక్సెస్ చేయడం
print("\nJSON ఫీల్డ్స్:")
print(f"టైటిల్: {json_result['title']}")
print(f"సంవత్సరం: {json_result['year']}")
print(f"దర్శకుడు: {json_result['director']}")
print(f"నటులు: {', '.join(json_result['actors'])}")
print(f"కథ: {json_result['plot'][:100]}...")
print(f"రేటింగ్: {json_result['rating']}/10")

### 5. PydanticOutputParser

`PydanticOutputParser` అనేది LLM ఔట్‌పుట్‌ని Pydantic మోడల్‌లోకి మార్చుతుంది, ఇది టైప్ చెకింగ్ మరియు వాలిడేషన్‌ని అందిస్తుంది:

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Pydantic మోడల్ డిఫైన్ చేయడం
class TeluguCricketer(BaseModel):
    name: str = Field(description="క్రికెటర్ పేరు")
    born_year: int = Field(description="పుట్టిన సంవత్సరం")
    teams: List[str] = Field(description="ఆడిన జట్లు")
    achievements: List[str] = Field(description="ముఖ్యమైన విజయాలు")
    stats: dict = Field(description="క్రికెట్ స్టాటిస్టిక్స్")
    bio: str = Field(description="చిన్న బయోగ్రఫీ")

# PydanticOutputParser క్రియేట్ చేయడం
pydantic_parser = PydanticOutputParser(pydantic_object=TeluguCricketer)

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
pydantic_prompt = ChatPromptTemplate.from_template(
    """తెలుగు క్రికెట్ ఆటగాడు {player_name} గురించి సమాచారం అందించండి.
    
    {format_instructions}
    """
)

# ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్‌ని పొందడం
format_instructions = pydantic_parser.get_format_instructions()

# చెయిన్ క్రియేట్ చేయడం
pydantic_chain = pydantic_prompt.partial(
    format_instructions=format_instructions
) | llm | pydantic_parser

# చెయిన్‌ని రన్ చేయడం
pydantic_result = pydantic_chain.invoke({"player_name": "VVS లక్ష్మణ్"})
print("PydanticOutputParser రిజల్ట్:")
print(f"టైప్: {type(pydantic_result)}")
print(f"మోడల్: {pydantic_result.__class__.__name__}")

# Pydantic మోడల్ ఫీల్డ్స్‌ని యాక్సెస్ చేయడం
print("\nPydantic మోడల్ ఫీల్డ్స్:")
print(f"పేరు: {pydantic_result.name}")
print(f"పుట్టిన సంవత్సరం: {pydantic_result.born_year}")
print(f"జట్లు: {', '.join(pydantic_result.teams)}")
print(f"విజయాలు: {', '.join(pydantic_result.achievements[:2])}...")
print(f"స్టాటిస్టిక్స్: {pydantic_result.stats}")
print(f"బయో: {pydantic_result.bio[:100]}...")

### 6. RegexParser

`RegexParser` అనేది రెగ్యులర్ ఎక్స్‌ప్రెషన్స్‌ని ఉపయోగించి LLM ఔట్‌పుట్‌ని పార్స్ చేస్తుంది:

In [ ]:
from langchain_core.output_parsers import RegexParser

# RegexParser క్రియేట్ చేయడం
regex_parser = RegexParser(
    regex=r"పేరు: (.*)\nపదవి: (.*)\nపార్టీ: (.*)",
    output_keys=["name", "position", "party"]
)

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
politician_prompt = ChatPromptTemplate.from_template(
    """తెలుగు రాజకీయ నాయకుడు {politician_name} గురించి సమాచారం అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని ఈ క్రింది ఫార్మాట్‌లో అందించండి:
    
    పేరు: [పూర్తి పేరు]
    పదవి: [ప్రస్తుత పదవి]
    పార్టీ: [రాజకీయ పార్టీ]
    """
)

# చెయిన్ క్రియేట్ చేయడం
politician_chain = politician_prompt | llm | regex_parser

# చెయిన్‌ని రన్ చేయడం
politician_result = politician_chain.invoke({"politician_name": "చంద్రబాబు నాయుడు"})
print("RegexParser రిజల్ట్:")
print(f"టైప్: {type(politician_result)}")
print(f"కంటెంట్: {politician_result}")

# పార్స్ చేసిన ఫీల్డ్స్‌ని యాక్సెస్ చేయడం
print("\nపార్స్ చేసిన ఫీల్డ్స్:")
print(f"పేరు: {politician_result['name']}")
print(f"పదవి: {politician_result['position']}")
print(f"పార్టీ: {politician_result['party']}")

## ఔట్‌పుట్ పార్సర్స్‌ని LCEL తో ఉపయోగించడం

LangChain Expression Language (LCEL) తో ఔట్‌పుట్ పార్సర్స్‌ని ఎలా ఉపయోగించాలో చూద్దాం:

In [ ]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
movie_prompt = ChatPromptTemplate.from_template(
    """తెలుగు సినిమా {movie_name} గురించి సమాచారం అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫీల్డ్స్‌తో:
    - title: సినిమా పేరు
    - year: విడుదల సంవత్సరం (నంబర్)
    - director: దర్శకుడు పేరు
    - actors: ప్రధాన నటుల జాబితా (అరే)
    - music_director: సంగీత దర్శకుడు
    - plot: కథ సారాంశం
    """
)

# JsonOutputParser క్రియేట్ చేయడం
json_parser = JsonOutputParser()

# LCEL చెయిన్ క్రియేట్ చేయడం
movie_chain = movie_prompt | llm | json_parser

# చెయిన్‌ని రన్ చేయడం
movie_result = movie_chain.invoke({"movie_name": "బాహుబలి"})
print("LCEL చెయిన్ రిజల్ట్:")
print(f"టైప్: {type(movie_result)}")
print(f"కంటెంట్: {movie_result}")

# JSON ఫీల్డ్స్‌ని యాక్సెస్ చేయడం
print("\nJSON ఫీల్డ్స్:")
print(f"టైటిల్: {movie_result['title']}")
print(f"సంవత్సరం: {movie_result['year']}")
print(f"దర్శకుడు: {movie_result['director']}")
print(f"నటులు: {', '.join(movie_result['actors'])}")
print(f"సంగీత దర్శకుడు: {movie_result['music_director']}")
print(f"కథ: {movie_result['plot'][:100]}...")

## ఔట్‌పుట్ పార్సర్స్‌తో ఎర్రర్ హ్యాండ్లింగ్

LLM ఔట్‌పుట్‌ని పార్స్ చేసేటప్పుడు ఎర్రర్స్ రావచ్చు. వాటిని ఎలా హ్యాండిల్ చేయాలో చూద్దాం:

In [ ]:
from langchain_core.output_parsers import OutputFixingParser

# ఎర్రర్‌ని సిమ్యులేట్ చేయడానికి ప్రాంప్ట్ క్రియేట్ చేయడం
bad_prompt = ChatPromptTemplate.from_template(
    """తెలుగు సినిమా {movie_name} గురించి సమాచారం అందించండి.
    
    ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్: సమాచారాన్ని JSON ఫార్మాట్‌లో అందించండి, ఈ క్రింది ఫీల్డ్స్‌తో:
    - title: సినిమా పేరు
    - year: విడుదల సంవత్సరం (నంబర్)
    - director: దర్శకుడు పేరు
    
    కానీ, మీ సమాధానం ముందు ఒక పరిచయం కూడా రాయండి.
    """
)

# JsonOutputParser క్రియేట్ చేయడం
json_parser = JsonOutputParser()

# OutputFixingParser క్రియేట్ చేయడం
fixing_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

# చెయిన్‌లు క్రియేట్ చేయడం
bad_chain = bad_prompt | llm | json_parser
fixing_chain = bad_prompt | llm | fixing_parser

try:
    # సాధారణ పార్సర్‌తో ప్రయత్నించడం
    bad_result = bad_chain.invoke({"movie_name": "పుష్ప"})
    print("సాధారణ పార్సర్ రిజల్ట్:")
    print(bad_result)
except Exception as e:
    print(f"సాధారణ పార్సర్ ఎర్రర్: {e}")

# OutputFixingParser తో ప్రయత్నించడం
fixing_result = fixing_chain.invoke({"movie_name": "పుష్ప"})
print("\nOutputFixingParser రిజల్ట్:")
print(fixing_result)

## కస్టమ్ ఔట్‌పుట్ పార్సర్ క్రియేట్ చేయడం

మనం స్వంత కస్టమ్ ఔట్‌పుట్ పార్సర్‌ని క్రియేట్ చేయవచ్చు:

In [ ]:
from langchain_core.output_parsers.base import BaseOutputParser

class TeluguMovieReviewParser(BaseOutputParser):
    """తెలుగు సినిమా రివ్యూలను పార్స్ చేసే కస్టమ్ ఔట్‌పుట్ పార్సర్."""
    
    def parse(self, text: str) -> dict:
        """టెక్స్ట్‌ని పార్స్ చేసి, స్ట్రక్చర్డ్ ఫార్మాట్‌లో రిటర్న్ చేస్తుంది."""
        lines = text.strip().split('\n')
        result = {}
        
        # టైటిల్‌ని పార్స్ చేయడం
        for line in lines:
            if "సినిమా పేరు" in line or "టైటిల్" in line:
                result["title"] = line.split(":")[-1].strip()
            elif "సంవత్సరం" in line or "విడుదల" in line:
                year_str = line.split(":")[-1].strip()
                # సంవత్సరాన్ని నంబర్‌గా మార్చడం
                try:
                    result["year"] = int(year_str)
                except ValueError:
                    result["year"] = year_str
            elif "దర్శకుడు" in line or "డైరెక్టర్" in line:
                result["director"] = line.split(":")[-1].strip()
            elif "రేటింగ్" in line:
                rating_str = line.split(":")[-1].strip()
                # రేటింగ్‌ని నంబర్‌గా మార్చడం
                try:
                    if "/" in rating_str:
                        num, den = rating_str.split("/")
                        result["rating"] = float(num.strip()) / float(den.strip()) * 10
                    else:
                        result["rating"] = float(rating_str)
                except ValueError:
                    result["rating"] = rating_str
        
        # సమీక్షను పార్స్ చేయడం
        review_lines = []
        review_started = False
        
        for line in lines:
            if "సమీక్ష" in line or "రివ్యూ" in line:
                review_started = True
                # హెడర్‌ని స్కిప్ చేయడం
                if ":" in line:
                    review_lines.append(line.split(":", 1)[1].strip())
                continue
            
            if review_started and line.strip() and not any(key in line for key in ["సినిమా పేరు", "టైటిల్", "సంవత్సరం", "విడుదల", "దర్శకుడు", "డైరెక్టర్", "రేటింగ్"]):
                review_lines.append(line.strip())
        
        result["review"] = " ".join(review_lines).strip()
        
        return result
    
    def get_format_instructions(self) -> str:
        """ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్‌ని రిటర్న్ చేస్తుంది."""
        return """సమాచారాన్ని ఈ క్రింది ఫార్మాట్‌లో అందించండి:
        
        సినిమా పేరు: [సినిమా పేరు]
        సంవత్సరం: [విడుదల సంవత్సరం]
        దర్శకుడు: [దర్శకుడు పేరు]
        రేటింగ్: [మీ రేటింగ్]/10
        
        సమీక్ష:
        [సినిమా గురించి మీ సమీక్ష ఇక్కడ రాయండి]
        """

# కస్టమ్ పార్సర్‌ని క్రియేట్ చేయడం
custom_parser = TeluguMovieReviewParser()

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
custom_prompt = ChatPromptTemplate.from_template(
    """తెలుగు సినిమా {movie_name} గురించి ఒక సమీక్ష రాయండి.
    
    {format_instructions}
    """
)

# ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్‌ని పొందడం
format_instructions = custom_parser.get_format_instructions()

# చెయిన్ క్రియేట్ చేయడం
custom_chain = custom_prompt.partial(
    format_instructions=format_instructions
) | llm | custom_parser

# చెయిన్‌ని రన్ చేయడం
custom_result = custom_chain.invoke({"movie_name": "RRR"})
print("కస్టమ్ పార్సర్ రిజల్ట్:")
print(f"టైప్: {type(custom_result)}")
print(f"కంటెంట్: {custom_result}")

# పార్స్ చేసిన ఫీల్డ్స్‌ని యాక్సెస్ చేయడం
print("\nపార్స్ చేసిన ఫీల్డ్స్:")
print(f"టైటిల్: {custom_result.get('title', 'N/A')}")
print(f"సంవత్సరం: {custom_result.get('year', 'N/A')}")
print(f"దర్శకుడు: {custom_result.get('director', 'N/A')}")
print(f"రేటింగ్: {custom_result.get('rating', 'N/A')}/10")
print(f"సమీక్ష: {custom_result.get('review', 'N/A')[:100]}...")

## రియల్-వరల్డ్ ఉదాహరణ: తెలుగు సినిమా అనాలిసిస్ టూల్

ఇప్పుడు, మనం ఔట్‌పుట్ పార్సర్స్‌ని ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూద్దాం: తెలుగు సినిమా అనాలిసిస్ టూల్.

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Optional

# Pydantic మోడల్స్ డిఫైన్ చేయడం
class MovieCharacter(BaseModel):
    name: str = Field(description="పాత్ర పేరు")
    actor: str = Field(description="నటించిన నటుడు/నటి పేరు")
    description: str = Field(description="పాత్ర వివరణ")
    importance: str = Field(description="కథలో పాత్ర ప్రాముఖ్యత (ప్రధాన/సహాయ/మైనర్)")

class MovieScene(BaseModel):
    description: str = Field(description="సీన్ వివరణ")
    characters: List[str] = Field(description="సీన్‌లో ఉన్న పాత్రలు")
    importance: int = Field(description="కథలో సీన్ ప్రాముఖ్యత (1-10 స్కేల్‌లో)")
    emotional_impact: str = Field(description="సీన్ యొక్క ఎమోషనల్ ఇంపాక్ట్")

class MovieAnalysis(BaseModel):
    title: str = Field(description="సినిమా పేరు")
    year: int = Field(description="విడుదల సంవత్సరం")
    director: str = Field(description="దర్శకుడు పేరు")
    genre: List[str] = Field(description="సినిమా జానర్స్")
    plot_summary: str = Field(description="కథ సారాంశం")
    main_characters: List[MovieCharacter] = Field(description="ప్రధాన పాత్రలు")
    key_scenes: List[MovieScene] = Field(description="కీలక సీన్స్")
    themes: List[str] = Field(description="సినిమాలో ప్రధాన థీమ్స్")
    technical_aspects: dict = Field(description="టెక్నికల్ అస్పెక్ట్స్ (సినిమాటోగ్రఫీ, సంగీతం, ఎడిటింగ్, మొదలైనవి)")
    box_office: Optional[str] = Field(description="బాక్స్ ఆఫీస్ కలెక్షన్స్")
    critical_reception: str = Field(description="విమర్శకుల స్పందన")
    cultural_impact: str = Field(description="సాంస్కృతిక ప్రభావం")
    overall_rating: float = Field(description="ఒవరాల్ రేటింగ్ (1-10 స్కేల్‌లో)")

# PydanticOutputParser క్రియేట్ చేయడం
movie_parser = PydanticOutputParser(pydantic_object=MovieAnalysis)

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
movie_analysis_prompt = ChatPromptTemplate.from_template(
    """మీరు తెలుగు సినిమా విశ్లేషకుడు. తెలుగు సినిమా {movie_name} గురించి లోతైన విశ్లేషణ చేయండి.
    
    ఈ విశ్లేషణలో ఈ క్రింది అంశాలు ఉండాలి:
    1. సినిమా పేరు, విడుదల సంవత్సరం, దర్శకుడు, జానర్
    2. కథ సారాంశం
    3. ప్రధాన పాత్రలు (కనీసం 3)
    4. కీలక సీన్స్ (కనీసం 3)
    5. ప్రధాన థీమ్స్
    6. టెక్నికల్ అస్పెక్ట్స్
    7. బాక్స్ ఆఫీస్ కలెక్షన్స్
    8. విమర్శకుల స్పందన
    9. సాంస్కృతిక ప్రభావం
    10. ఒవరాల్ రేటింగ్
    
    {format_instructions}
    """
)

# ఫార్మాట్ ఇన్‌స్ట్రక్షన్స్‌ని పొందడం
format_instructions = movie_parser.get_format_instructions()

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# చెయిన్ క్రియేట్ చేయడం
movie_analysis_chain = movie_analysis_prompt.partial(
    format_instructions=format_instructions
) | llm | movie_parser

# చెయిన్‌ని రన్ చేయడం
movie_analysis = movie_analysis_chain.invoke({"movie_name": "బాహుబలి"})

# విశ్లేషణను ప్రింట్ చేయడం
print(f"సినిమా విశ్లేషణ: {movie_analysis.title} ({movie_analysis.year})")
print(f"దర్శకుడు: {movie_analysis.director}")
print(f"జానర్: {', '.join(movie_analysis.genre)}")
print("\nకథ సారాంశం:")
print(movie_analysis.plot_summary)

print("\nప్రధాన పాత్రలు:")
for i, character in enumerate(movie_analysis.main_characters):
    print(f"{i+1}. {character.name} ({character.actor}) - {character.importance}")
    print(f"   {character.description}")

print("\nకీలక సీన్స్:")
for i, scene in enumerate(movie_analysis.key_scenes):
    print(f"{i+1}. ప్రాముఖ్యత: {scene.importance}/10 - ఎమోషనల్ ఇంపాక్ట్: {scene.emotional_impact}")
    print(f"   పాత్రలు: {', '.join(scene.characters)}")
    print(f"   వివరణ: {scene.description}")

print("\nప్రధాన థీమ్స్:")
for i, theme in enumerate(movie_analysis.themes):
    print(f"{i+1}. {theme}")

print("\nటెక్నికల్ అస్పెక్ట్స్:")
for aspect, description in movie_analysis.technical_aspects.items():
    print(f"{aspect}: {description}")

print(f"\nబాక్స్ ఆఫీస్: {movie_analysis.box_office}")
print(f"విమర్శకుల స్పందన: {movie_analysis.critical_reception}")
print(f"సాంస్కృతిక ప్రభావం: {movie_analysis.cultural_impact}")
print(f"\nఒవరాల్ రేటింగ్: {movie_analysis.overall_rating}/10")

## ముగింపు

ఈ ట్యుటోరియల్‌లో, మనం LangChain లో ఔట్‌పుట్ పార్సర్స్ గురించి నేర్చుకున్నాము:

1. ఔట్‌పుట్ పార్సర్స్ అంటే ఏమిటో మరియు వాటి ప్రాముఖ్యత గురించి తెలుసుకున్నాము
2. LangChain లో అందుబాటులో ఉన్న వివిధ రకాల ఔట్‌పుట్ పార్సర్స్‌ని చూశాము:
   - StrOutputParser
   - ListOutputParser
   - CommaSeparatedListOutputParser
   - JsonOutputParser
   - PydanticOutputParser
   - RegexParser
3. ఔట్‌పుట్ పార్సర్స్‌ని LCEL తో ఎలా ఉపయోగించాలో నేర్చుకున్నాము
4. ఔట్‌పుట్ పార్సర్స్‌తో ఎర్రర్ హ్యాండ్లింగ్‌ని చూశాము
5. కస్టమ్ ఔట్‌పుట్ పార్సర్‌ని ఎలా క్రియేట్ చేయాలో నేర్చుకున్నాము
6. ఔట్‌పుట్ పార్సర్స్‌ని ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూశాము: తెలుగు సినిమా అనాలిసిస్ టూల్

ఔట్‌పుట్ పార్సర్స్ అనేవి LLM అప్లికేషన్‌లలో చాలా ముఖ్యమైన కంపోనెంట్స్, ఎందుకంటే అవి LLM ఔట్‌పుట్‌ని స్ట్రక్చర్డ్ ఫార్మాట్‌లోకి మార్చడానికి సహాయపడతాయి, ఇది ప్రోగ్రామాటిక్‌గా యాక్సెస్ చేయడానికి మరియు ఇతర సిస్టమ్స్‌తో ఇంటిగ్రేట్ చేయడానికి సులభం.

తదుపరి ట్యుటోరియల్‌లో, మనం LangChain లో మెమరీ కాన్సెప్ట్స్ గురించి నేర్చుకుంటాము.